In [1]:
import json 

with open('레지던스_reviews.json','r',encoding='utf-8')as f:
    review_list = json.load(f)

In [2]:
good_cnt, bad_cnt = 0, 0

for review in review_list:
    if review['stars'] == 5:
        good_cnt += 1
    else:
        bad_cnt += 1

In [3]:
reviews_good, reviews_bad = [], []

for review in review_list:
    if review['stars'] == 5:
        reviews_good.append('[REVIEW_START]' + review['review'] + '[REVIEW_END]')
    else:
        reviews_bad.append('[REVIEW_START]' + review['review'] + '[REVIEW_END]')

In [4]:
import datetime
from dateutil import parser

def preprocess_reviews(path='./res/reviews.json'):
    with open(path, 'r', encoding='UTF8') as f:
        review_list = json.load(f)
    
    reviews_good, reviews_bad = [], []

    current_date = datetime.datetime.now()
    date_boundary = current_date - datetime.timedelta(days=6*30)

    for review in review_list:
        review_date_str = review['date']
        try:
            review_date = parser.parse(review_date_str)
        except (ValueError, TypeError):
            review_date = current_date
        
        if review_date < date_boundary:
            continue

        if review['stars'] == 5:
            reviews_good.append('[REVIEW_START]' + review['review'] + '[REVIEW_END]')
        else:
            reviews_bad.append('[REVIEW_START]' + review['review'] + '[REVIEW_END]')

    review_good_text = '\n'.join(reviews_good)
    review_bad_text = '\n'.join(reviews_bad)

    return review_good_text, review_bad_text

good, bad = preprocess_reviews("레지던스_reviews.json")

In [5]:
PROMPT_BASELINE = f"""아래 숙소에 대해서 3문장 내로 요약해줘:"""

In [6]:
from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv()

True

In [7]:
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

def summarize(reviews, prompt, temperature=0.0, model='gpt-3.5-turbo'):
    prompt = prompt + '\n\n' + reviews

    completion = client.chat.completions.create(
        model=model,
        messages=[{'role': 'user', 'content': prompt}],
        temperature=temperature
    )

    return completion

## ZERO - SHOT

In [ ]:
prompt = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다.

요약 결과는 다음 조건들을 충족해야 합니다:
1. 모든 문장은 항상 존댓말로 끝나야 합니다.
2. 숙소에 대해 소개하는 톤앤매너로 작성해주세요.
  2-1. 좋은 예시
    a) 전반적으로 좋은 숙소였고 방음도 괜찮았다는 평입니다.
    b) 재방문 예정이라는 평들이 존재합니다.
  2-2. 나쁜 예시
    a) 좋은 숙소였고 방음도 괜찮았습니다.
    b) 재방문 예정입니다.
3. 요약 결과는 최소 2문장, 최대 5문장 사이로 작성해주세요. 숫자로 나열하지 않고 자연스럽게 이어주세요.
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

print(f"좋은 리뷰 요약: {summarize(good, prompt).choices[0].message.content}")
print(f"나쁜 리뷰 요약: {summarize(bad, prompt).choices[0].message.content}")

## ONE - SHOT

In [ ]:
summary_good = summarize(good, prompt, temperature=0.0, model='gpt-4o-2024-05-13').choices[0].message.content
summary_bad = summarize(bad, prompt, temperature=0.0, model='gpt-4o-2024-05-13').choices[0].message.content

In [ ]:
prompt_1shot = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다.

요약 결과는 다음 조건들을 충족해야 합니다:
1. 모든 문장은 항상 존댓말로 끝나야 합니다.
2. 숙소에 대해 소개하는 톤앤매너로 작성해주세요.
  2-1. 좋은 예시
    a) 전반적으로 좋은 숙소였고 방음도 괜찮았다는 평입니다.
    b) 재방문 예정이라는 평들이 존재합니다.
  2-2. 나쁜 예시
    a) 좋은 숙소였고 방음도 괜찮았습니다.
    b) 재방문 예정입니다.
3. 요약 결과는 최소 2문장, 최대 5문장 사이로 작성해주세요. 숫자로 나열하지 않고 자연스럽게 이어주세요.

다음은 요약 예시입니다.
예시 요약 결과:
{summary_good}

아래 숙소 리뷰들에 대해 요약해주세요:"""

print(f"좋은 리뷰 요약: {summarize(good, prompt_1shot).choices[0].message.content}")
print(f"나쁜 리뷰 요약: {summarize(bad, prompt_1shot).choices[0].message.content}")

## TWO - SHOT

In [ ]:
prompt_2shot = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다.

요약 결과는 다음 조건들을 충족해야 합니다:
1. 모든 문장은 항상 존댓말로 끝나야 합니다.
2. 숙소에 대해 소개하는 톤앤매너로 작성해주세요.
  2-1. 좋은 예시
    a) 전반적으로 좋은 숙소였고 방음도 괜찮았다는 평입니다.
    b) 재방문 예정이라는 평들이 존재합니다.
  2-2. 나쁜 예시
    a) 좋은 숙소였고 방음도 괜찮았습니다.
    b) 재방문 예정입니다.
3. 요약 결과는 최소 2문장, 최대 5문장 사이로 작성해주세요. 숫자로 나열하지 않고 자연스럽게 이어주세요.

다음은 요약 예시입니다.
좋은 리뷰 요약 결과:
{summary_good}
나쁜 리뷰 요약 결과:
{summary_bad}

아래 숙소 리뷰들에 대해 요약해주세요:"""

print(f"좋은 리뷰 요약: {summarize(good, prompt_2shot).choices[0].message.content}")
print(f"나쁜 리뷰 요약: {summarize(bad, prompt_2shot).choices[0].message.content}")